# ஆட்டோஜென் கருவி பயன்பாட்டு உதாரணம்


## தேவையான தொகுப்புகளை இறக்குமதி செய்யவும்


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## கிளையன்ட் உருவாக்குதல்

இந்த மாதிரியில், LLM-ஐ அணுக [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) பயன்படுத்தப்படும்.

`model` `gpt-4o-mini` என வரையறுக்கப்பட்டுள்ளது. GitHub Models சந்தையில் கிடைக்கும் மற்றொரு மாடலுக்கு மாறி, வெவ்வேறு முடிவுகளைப் பார்க்க முயற்சிக்கவும்.

ஒரு விரைவான சோதனையாக, நாம் ஒரு எளிய கேள்வியை இயக்குவோம் - `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## செயல்பாடுகளை வரையறுத்தல்

இந்த எடுத்துக்காட்டில், முகவருக்கு கிடைக்கும் விடுமுறை இடங்களின் பட்டியல் மற்றும் அவற்றின் கிடைப்புகளை கொண்ட ஒரு செயல்பாட்டை ஒரு கருவியாக அணுக அனுமதி வழங்கப்படும்.

இதை ஒரு பயண முகவர், உதாரணமாக, ஒரு பயண தரவுத்தொகுப்பை அணுகும் சூழல் எனக் கருதலாம்.

இந்த மாதிரியைப் பார்வையிடும் போது, முகவர் அழைக்கக்கூடிய புதிய செயல்பாடுகள் மற்றும் கருவிகளை வரையறுக்க முயற்சிக்கலாம்.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## செயல்பாட்டு கருவியை வரையறுத்தல்  
`vacation_destinations` ஐ `FunctionTool` ஆக பயன்படுத்துவதற்கு, அதை ஒன்றாக வரையறுக்க வேண்டும்.  

மேலும், பயனர் கேட்டுள்ள பணிக்கான தொடர்பில் அந்த கருவி எதற்காக பயன்படுத்தப்படுகிறது என்பதை முகவர் அடையாளம் காண உதவியாக, கருவியின் விளக்கத்தையும் வழங்குவோம்.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## ஏஜென்டை வரையறுத்தல்

இப்போது கீழே உள்ள குறியீட்டில் ஏஜென்டை உருவாக்கலாம். பயண இடங்களை கண்டறிய பயனர்களுக்கு உதவ ஏஜென்டுக்கு வழிகாட்டுதல்களை வழங்க `system_message`-ஐ வரையறுக்கிறோம்.

மேலும், `reflect_on_tool_use` அளவுருவை true ஆக அமைக்கிறோம். இது கருவி அழைப்பின் பதிலை LLM மூலம் எடுத்துக்கொண்டு இயற்கை மொழியில் பதிலை அனுப்ப உதவுகிறது.

இந்த அளவுருவை false ஆக அமைத்து வித்தியாசத்தை பார்க்கலாம்.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## ஏஜென்டை இயக்குதல்

இப்போது, டோக்கியோக்கு பயணம் செய்ய வேண்டும் என்று தொடக்க பயனர் செய்தியுடன் ஏஜென்டை இயக்கலாம்.

நீங்கள் இந்த நகர இலக்கை மாற்றி, அந்த நகரத்தின் கிடைக்கும் நிலைமைக்கு ஏஜென்டின் பதிலை எப்படி இருக்கிறது என்பதை பார்க்கலாம்.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
